### Symbolic Asynchronous Graph

To further explore the network dynamics, we can use its symbolic asynchronous graph. Although, this feature will mostly be used for building standalone algorithms and not directly used within a notebook.

In [1]:
try:
    graph = SymbolicAsyncGraph(g2a)
except Exception as e:
    # The construction of a graph can fail if there are inconsistencies between the
    # regulation properties and the update functions of the network.
    print(e)

# There are 32 states and 2 colors (two possible instantiations of function `f1`)
print(int(graph.unit_vertices().cardinality()), "x", int(graph.unit_colors().cardinality()))

name 'SymbolicAsyncGraph' is not defined


NameError: name 'graph' is not defined

Here, the constructor will check whether the Boolean network contains some inconsistencies: for example, an unused parameter, unused observable regulation, or update functions that do not satisfy the regulation constraints (activation in place of inhibition, etc.).

The graph operates using three types of sets: `ColorSet`, `VertexSet` and `ColoredVertexSet`. A color set contains possible valuations of logical parameters and uninterpreted functions. So, each member of a color set fully specifies one exact Boolean network. If there are no parameters, this set can be only "empty" or "1". Similarly, a vertex set contains vertices of the graph. Finally, a colored vertex set contains pairs of colors and vertices, meaning that for each color, we can have a different set of vertices (or, similarly, for each vertex a different set of colors). This type of relation can be then used to represent a set of vertices that is different for individual parametrisations.

In [ ]:
a_color = graph.unit_colors().pick_singleton()
a_color.cardinality()

A set of vertices can be also iterated through, but keep in mind that the number of vertices can be huge for large models. Always check the size of your set using `cardinality` first.

In [ ]:
vertices = graph.fix_variable("SciP", False).vertices()
for v in vertices.iterator():
    print(v)

We can also create a vertex singleton directly (this will still admit every possible color though!):

In [ ]:
singleton = graph.fix_vertex([True, False, False, True, False]) # The result is a ColoredVertexSet
print(singleton.vertices().cardinality())
print(singleton.colors().cardinality())

Finally, we can use the graph to compute successors and predecessors for colored sets of vertices. Using this operation, we can implement more complex algorithms, like SCC decomposition or safety analysis.

In [ ]:
reachable = singleton
for i in range(10):
    step = graph.post(reachable)
    new = step.minus(reachable)
    print(i+1, "Discovered", new.vertices().cardinality(), "using colors", new.colors().cardinality())
    if new.is_empty():
        break
    reachable = reachable.union(new)

print("Total discovered vertices:", reachable.vertices().cardinality())

It is also possible to use `var_post` and `var_pre` to only update a specific variable in the network during a transition. This is typically much more efficient (but of course, you need to perform more updates to cover all network variables). Furthermore, the library provides other specialized alternatives to `post` and `pre` which, for example, efficiently compute successors/predecessors within a specific set, etc.

To save the results of the computation, we can use the fact that these sets are all internally represented as BDDs, which we can dump to a string, or visualized as a `.dot` file:

In [ ]:
reachable_string = reachable.to_bdd().to_raw_string()
reachable_string

In [ ]:
import graphviz

graphviz.Source(reachable.to_bdd().to_dot())

In [ ]:
# We need to copy an existing set to preserve all metadata about the model,
# since these are not saved in the BDD
reachable_reloaded = ColoredVertexSet(graph, Bdd.from_raw_string(reachable_string))

# r1 <=> r2 must be a tautology if the sets are equivalent:
reachable_reloaded.to_bdd().l_iff(reachable.to_bdd()).is_true()

However, keep in mind that the raw string does not contain any information about the underlying model, and you cannot mix BDD representations between different models! So always make sure not to mix BDD files for different models.